<a href="https://colab.research.google.com/github/opeeeeeeeeeeeeeeemi/Data-Engineering-Projects/blob/main/Data_Transfer_automation_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import mysql.connector
import psycopg2

# MySQL connection details
mysql_user = 'root'
mysql_password = '2OfoPPjFTJ8Y2BkENHhqY6AZ'
mysql_host = '172.21.42.30'
mysql_database = 'sales'

# PostgreSQL connection details
dsn_hostname = '127.0.0.1'
dsn_user = 'postgres'
dsn_pwd = 'MTY4ODEtb3BleWVt'
dsn_port = "5432"
dsn_database = "postgres"

# Function to get the last rowid from the PostgreSQL data warehouse
def get_last_rowid():
    try:
        conn = psycopg2.connect(
            database=dsn_database,
            user=dsn_user,
            password=dsn_pwd,
            host=dsn_hostname,
            port=dsn_port
        )
        cursor = conn.cursor()
        cursor.execute("SELECT MAX(rowid) FROM sales_data")
        last_row_id = cursor.fetchone()[0]
        cursor.close()
        conn.close()
        return last_row_id if last_row_id is not None else 0
    except Exception as e:
        print(f"Error fetching last row ID: {e}")
        return None

# Function to get the latest records from the MySQL database
def get_latest_records(last_row_id):
    try:
        connection = mysql.connector.connect(
            user=mysql_user,
            password=mysql_password,
            host=mysql_host,
            database=mysql_database
        )
        cursor = connection.cursor()
        query = "SELECT * FROM sales_data WHERE rowid > %s"
        cursor.execute(query, (last_row_id,))
        new_records = cursor.fetchall()
        cursor.close()
        connection.close()
        return new_records
    except Exception as e:
        print(f"Error fetching latest records: {e}")
        return []

# Function to insert new records into the PostgreSQL data warehouse
def insert_records(records):
    if not records:
        print("No new records to insert.")
        return

    try:
        conn = psycopg2.connect(
            database=dsn_database,
            user=dsn_user,
            password=dsn_pwd,
            host=dsn_hostname,
            port=dsn_port
        )
        cursor = conn.cursor()
        insert_query = """
            INSERT INTO sales_data (rowid, cproduct,category)
            VALUES ("Television","Electronics"), ("Laptop","Electronics"), ("Mobile","Electronics")
        """
        cursor.executemany(insert_query, records)
        conn.commit()
        cursor.close()
        conn.close()
        print(f"New rows inserted into production data warehouse = {len(records)}")
    except Exception as e:
        print(f"Error inserting records: {e}")

# Main program execution flow
last_row_id = get_last_rowid()
print("Last row id on production data warehouse = ", last_row_id)

new_records = get_latest_records(last_row_id)
print("New rows on staging data warehouse = ", len(new_records))

insert_records(new_records)

# No explicit disconnection needed here as each function handles its own connection closure

